# Oracle Spacial Data 추가하기

In [11]:
# Import necessary libraries
import os
import cx_Oracle
import logging
import zipfile
import shutil
import geopandas as gpd

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

def get_user_choice(zip_files):
    logger.debug("get_user_choice")
    while True:
        logger.debug("True")
        try:
            logger.debug("try")
            choice = input("Enter the number of the ZIP file you want to process (or 'q' to quit): ")
            if choice.lower() == 'q':
                return None
            choice = int(choice)
            if 1 <= choice <= len(zip_files):
                return zip_files[choice - 1]
            else:
                logger.debug(f"Please enter a number between 1 and {len(zip_files)}")
        except ValueError:
            logger.debug("Please enter a valid number or 'q' to quit.")

def load_shape():
    current_directory = os.getcwd()    
    uploads = os.path.join(current_directory, 'uploads')

    zip_files = [f for f in os.listdir(uploads) if f.endswith('.zip')]
    
    if not zip_files:
        logger.debug("No ZIP files found in the uploads directory.")
        return
    
    logger.debug("Available ZIP files:")
    for i, file in enumerate(zip_files, 1):
        print(f"{i}. {file}")
    
    selected_zip = get_user_choice(zip_files)
    if selected_zip is None:
        logger.debug("Operation cancelled by user.")
        return

    zip_path = os.path.join(uploads, selected_zip)
    logger.debug(f"Selected ZIP file: {selected_zip}")

    # 임시 디렉토리 생성
    temp_dir = os.path.join(uploads, 'temp_shapefile')
    os.makedirs(temp_dir, exist_ok=True)

    try:
        # ZIP 파일 압축 해제
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        
        # .shp 파일 찾기
        shp_file = None
        for filename in os.listdir(temp_dir):
            if filename.endswith('.shp'):
                shp_file = os.path.join(temp_dir, filename)
                break
        
        if not shp_file:
            raise ValueError("No .shp file found in the ZIP archive")
        
        # Shape 파일 처리 및 Oracle DB에 저장
        result = process_shapefile(shp_file)
        logger.debug(result)
    
    except Exception as e:
        logger.debug(f"Error processing file: {str(e)}")
    
    finally:
        # 임시 디렉토리 삭제
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

# Define function to process a shapefile
def process_shapefile(filename):
    try:
        # Shape 파일 읽기
        gdf = gpd.read_file(filename)

        # Oracle 연결
        connection = get_db_connection()
        cursor = connection.cursor()

        # 인덱스 비활성화
        cursor.execute("ALTER INDEX MY_SHAPE_SPATIAL_IDX UNUSABLE")
        
        # 데이터 삽입
        inserted_count = 0
        for index, row in gdf.iterrows():
            geom = row["geometry"]
            name = row.get("name", f"Feature_{index}")
            logger.debug(f'geom : {geom}, name : {name}')

            if geom.geom_type == 'Point':
                # Point 좌표 추출
                x, y = geom.x, geom.y

                insert_query = """
                INSERT INTO my_shape_table (id, name, geometry)
                VALUES (my_shape_seq.NEXTVAL, :name, 
                        SDO_GEOMETRY(
                            2001,  -- 2D Point
                            4326,  -- SRID for WGS84
                            SDO_POINT_TYPE(:x, :y, NULL),
                            NULL, NULL
                        )
                )
                """
                cursor.execute(insert_query, name=name, x=x, y=y)
                inserted_count += 1

            elif geom.geom_type == 'LineString':
                # LineString 좌표 추출
                coords = [(point[0], point[1]) for point in geom.coords]
                sdo_ordinates = ", ".join([f"{x}, {y}" for x, y in coords])
                print(f'sdo_ordinates of line : {sdo_ordinates}')

                insert_query = f"""
                INSERT INTO my_shape_table (id, name, geometry)
                VALUES (my_shape_seq.NEXTVAL, :name, 
                        SDO_GEOMETRY(
                            2002,  -- 2D LineString
                            4326,  -- SRID for WGS84
                            NULL,
                            SDO_ELEM_INFO_ARRAY(1, 2, 1),
                            SDO_ORDINATE_ARRAY({sdo_ordinates})
                        )
                )
                """
                cursor.execute(insert_query, name=name)
                inserted_count += 1

            elif geom.geom_type == 'Polygon':
                # Polygon 좌표 추출
                exterior_coords = [(point[0], point[1]) for point in geom.exterior.coords]
                sdo_ordinates = ", ".join([f"{x}, {y}" for x, y in exterior_coords])

                insert_query = f"""
                INSERT INTO my_shape_table (id, name, geometry)
                VALUES (my_shape_seq.NEXTVAL, :name, 
                        SDO_GEOMETRY(
                            2003,  -- 2D Polygon
                            4326,  -- SRID for WGS84
                            NULL,
                            SDO_ELEM_INFO_ARRAY(1, 1003, 1),
                            SDO_ORDINATE_ARRAY({sdo_ordinates})
                        )
                )
                """
                cursor.execute(insert_query, name=name)
                inserted_count += 1

            else:
                logger.debug(f'Unsupported geometry type: {geom.geom_type}')


        # 인덱스 활성화
        cursor.execute("ALTER INDEX MY_SHAPE_SPATIAL_IDX REBUILD")

        # 인덱스 통계 수집
        cursor.execute("BEGIN DBMS_STATS.GATHER_INDEX_STATS(USER, 'MY_SHAPE_SPATIAL_IDX'); END;")

        return f"Inserted {inserted_count} features into the database."
    except cx_Oracle.Error as error:
        logger.debug(f"Oracle Error occurred: {error}")
        return f"Error: {error}"
    except Exception as e:
        logger.debug(f"An unexpected error occurred: {e}")
        return f"Error: {e}"
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'connection' in locals():
            connection.close()

def get_db_connection():
    try:
        logger.info("trying to connect to Oracle")
        host = os.getenv("ORACLE_HOST")
        port = os.getenv("ORACLE_PORT")
        service_name = os.getenv("ORACLE_SERVICE_NAME")
        user = os.getenv("ORACLE_USER")
        password = os.getenv("ORACLE_PASSWORD")

        logger.debug(f"connection info : {host}, {port}, {service_name}")

        dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
        logger.debug(f"dsn : {dsn}")

        connection = cx_Oracle.connect(user=user, password=password, dsn=dsn)
        return connection
    except cx_Oracle.Error as error:
        logger.error(f"Error connecting to Oracle: {error}")
        raise

if __name__ == "__main__":
    load_shape()

DEBUG:__main__:Available ZIP files:
DEBUG:__main__:get_user_choice
DEBUG:__main__:True
DEBUG:__main__:try


1. crossings.zip
2. dam.zip
3. line.zip
4. rect.zip


DEBUG:__main__:Selected ZIP file: line.zip
DEBUG:fiona._env:GDAL_DATA found in environment.
DEBUG:fiona._env:PROJ_LIB found in environment.
DEBUG:fiona._env:GDAL_DATA found in environment.
DEBUG:fiona._env:PROJ_LIB found in environment.
ERROR:fiona._env:PROJ: proj_identify: C:\Program Files\PostgreSQL\10\share\contrib\postgis-3.1\proj\proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
DEBUG:fiona.ogrext:Got coordinate system
DEBUG:fiona.ogrext:Got coordinate system
ERROR:fiona._env:PROJ: proj_identify: C:\Program Files\PostgreSQL\10\share\contrib\postgis-3.1\proj\proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
DEBUG:fiona.ogrext:OLC_FASTSETNEXTBYINDEX: 1
DEBUG:fiona.ogrext:OLC_FASTFEATURECOUNT: 1
DEBUG:fiona.ogrext:Next index: 0
DEBUG:fiona.ogrext:Next index: 1
DEBUG:fiona.collection:Flushed buffer
DEBUG:fiona.collection:Stopped sess

sdo_ordinates of line : 127.10887055485821, 36.285883753349225, 127.10801224797345, 36.29563835497813


DEBUG:__main__:Inserted 1 features into the database.
